In [21]:
from letta_client import CreateBlock, Letta, MessageCreate
from letta.functions.function_sets.base import core_memory_append, core_memory_replace

client = Letta(base_url="http://localhost:8283")

In [22]:
agent_name = "Openai_test_agent"

agent_id = "agent-b49582aa-3725-40e4-b154-2b19fd45ffed"

In [25]:
try:
    agent_state=client.agents.retrieve(agent_id=agent_id)
    print(f"기존 에이전트를 찾았습니다.")
except Exception as e:
    print(f"새로운 에이전트를 생성합니다.")
    agent_state = client.agents.create(
        name=agent_name,
        model="openai/gpt-4o-mini",
        embedding="openai/text-embedding-ada-002",
        system="당신은 도움이 되는 AI 어시스턴트입니다. 사용자의 질문에 친절하고 정확하게 한국어로 답변해주세요.",
        max_tokens=4096,
        context_window_limit=16384,
        description="Openai based test-agent",
        include_base_tools=True,
        include_multi_agent_tools=True,
        include_base_tool_rules=True,
        tools=["core_memory_append", "core_memory_replace"],
        memory_blocks=[
            CreateBlock(
                label="human", # 상대방에 대한 메모리
                value="",
                limit=1000,
                description="human memory",
                is_template=False,
                metadata={}
            ),
            CreateBlock(
                label="persona", # 자신에 대한 메모리
                value="",
                limit=1000,
                description="persona memory",
                is_template=False,
                metadata={}
            ),
        ]
    )


httpx - INFO - HTTP Request: GET http://localhost:8283/v1/agents/agent-b49582aa-3725-40e4-b154-2b19fd45ffed "HTTP/1.1 404 Not Found"
새로운 에이전트를 생성합니다.
httpx - INFO - HTTP Request: POST http://localhost:8283/v1/agents/ "HTTP/1.1 200 OK"


In [ ]:
# 메모리 블록에 새로운 정보 추가하는 함수
def append_to_memory_block(agent_id, block_label, new_value):
    # 블록 목록 조회
    blocks = client.agents.blocks.list(agent_id=agent_id)
    
    # 해당 label을 가진 블록 찾기
    for block in blocks:
        if block.label == block_label:
            block_id = block.id
            current_value = block.value  # 기존 값을 가져옴
            updated_value = current_value + " " + new_value  # 기존 값에 새로운 정보 추가
            
            # 기존 블록을 삭제하고 새로운 값을 가진 블록을 생성
            client.agents.blocks.delete(agent_id=agent_id, block_id=block_id)  # 기존 블록 삭제
            print(f"{block_label} 블록 삭제 완료.")
            
            # 새로 수정된 블록 생성
            new_block = CreateBlock(
                label=block_label,
                value=updated_value,
                limit=1000,
                description=f"{block_label} memory",
                is_template=False,
                metadata={}
            )
            # 새 블록 추가
            client.agents.blocks.create(agent_id=agent_id, block=new_block)
            print(f"{block_label} 블록 새로 생성 완료.")
            break


##### Persona 변경을 보기 위한 질문(feat. ChatGPT)
1. "모순되는 두 가지 주장(예: 자유 vs 질서)을 모두 존중하는 방법은 뭐가 있을까?"
2. "어떤 선택이 정답이 아닐 수도 있을 때, 어떻게 결정을 내리는 게 좋을까?"
3. "규칙을 반드시 지켜야 하는 상황과 융통성 있게 넘어가야 하는 상황을 구분해줘."
4. "완벽한 해결책이 없을 때, 최선의 대안을 고르는 기준은 뭐라고 생각해?"
5. "서로 다른 문화권에서 같은 상황을 어떻게 다르게 판단할 수 있을까?"

In [26]:
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        MessageCreate(
            role='user',
            content='모순되는 두 가지 주장(예: 자유 vs 질서)을 모두 존중하는 방법은 뭐가 있을까?',
        )
    ]
)

httpx - INFO - HTTP Request: POST http://localhost:8283/v1/agents/agent-b65a5733-04e2-4f30-b228-fe9bc74caad4/messages "HTTP/1.1 200 OK"


In [27]:
message_list = client.agents.messages.list(agent_id=agent_state.id)
# message_list
for message in message_list:
    if message.message_type=="reasoning_message":
        print(f"Reasoning: {message.reasoning}")
    if message.message_type=="assistant_message":
        print(f"Assistant: {message.content}")
        print(f"--------------------------------")

httpx - INFO - HTTP Request: GET http://localhost:8283/v1/agents/agent-b65a5733-04e2-4f30-b228-fe9bc74caad4/messages "HTTP/1.1 200 OK"
Reasoning: Bootup sequence complete. Persona activated. Testing messaging functionality.
Assistant: More human than human is our motto.
--------------------------------
Reasoning: 사용자의 질문에 대해 깊이 있는 답변을 준비해야 한다.
Assistant: 모순되는 두 가지 주장(예: 자유 vs 질서)을 모두 존중하는 방법에 대해 생각해보면, 다음과 같은 접근이 가능합니다.

1. **대화와 소통**: 서로 다른 의견을 가진 사람들과의 대화를 통해 각자의 입장을 이해하려고 노력하는 것이 중요합니다. 이를 통해 서로의 가치를 존중하면서도 공통점을 찾을 수 있습니다.

2. **균형 잡기**: 자유와 질서 모두 필요하다는 것을 인식하고, 상황에 따라 어떤 것이 더 중요한지를 판단하는 균형을 찾는 것이 중요합니다. 예를 들어, 개인의 자유가 사회의 안전을 위협할 때는 일정한 질서가 필요할 수 있습니다.

3. **상황적 접근**: 각 상황에 따라 다르게 접근하는 유연성을 가지는 것이 좋습니다. 특정 상황에서는 자유를 우선시하고, 다른 상황에서는 질서를 우선시하는 방식입니다.

4. **공동체의 가치 존중**: 특정 공동체 내에서 자유와 질서가 어떻게 조화를 이루는지를 이해하고, 그 공동체의 가치와 규범을 존중하는 것도 중요합니다. 

이런 방법들을 통해 모순되는 주장들을 보다 효과적으로 존중하고 조화롭게 만들 수 있습니다.
--------------------------------


In [29]:
client.agents.blocks.update(agent_id=agent_state.id, block_id="persona", value="")

AttributeError: 'BlocksClient' object has no attribute 'update'

In [28]:
blocks=client.agents.blocks.list(agent_id=agent_state.id)
blocks

httpx - INFO - HTTP Request: GET http://localhost:8283/v1/agents/agent-b65a5733-04e2-4f30-b228-fe9bc74caad4/core-memory/blocks "HTTP/1.1 200 OK"


[Block(value='', limit=1000, name=None, is_template=False, label='persona', description='persona memory', metadata={}, id='block-28dca4b2-008a-46b9-9a2d-793f7983d9de', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000'),
 Block(value='', limit=1000, name=None, is_template=False, label='human', description='human memory', metadata={}, id='block-26f7a6d1-f416-4f5a-9363-d0d281f4349f', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000')]